# Calcul d'entropie des caractéristiques
***
**Authors**: BAIM Mohamed, MHAJAR Youness, BEZZAZ Ilias, GUERIN Marius, BANANI Amine, CHAQUIQ ELBADRE Youssef

In [42]:
# Libraries 
import os
import pandas as pd
from scipy.stats import entropy
import numpy as np
import ast 

Lors de l'entraînement, nous avons extrait diverses caractéristiques de chaque image pour entraîner notre modèle. Ces caractéristiques ont été enregistrées dans un fichier CSV pour ensuite calculer l'entropie des caractéristiques et ainsi les évaluer.

In [55]:
# Récupère les caracteriiqtiques extraites de chaque image 

df = pd.read_csv('dataset_caracteristiques.csv')
df.head()

,Picture Path,Label,width,height,Luminance,RGB,LBP,HSV
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Nuageux,600,400,0.178594,"[126.34504885660581, 126.34504885660581, 126.3...","[451, 2006, 1591, 15454, 34446, 33885, 9137, 6...","[0.0, 0.0]"
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Nuageux,271,186,0.118810,"[152.9002701495305, 169.04855283246093, 198.54...","[189, 1748, 894, 11282, 31090, 39998, 7514, 65...","[109.50477081542171, 60.599630809689984]"
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Nuageux,259,194,0.134323,"[150.96056491174835, 153.7316055129364, 155.91...","[816, 3787, 2685, 13576, 36313, 31807, 7939, 5...","[92.744845664964, 54.62629174463494]"
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Nuageux,338,149,0.126442,"[171.27977071005918, 177.1194305365903, 200.47...","[285, 1755, 1077, 13818, 29452, 38865, 8806, 7...","[123.27952258129773, 40.92878335777744]"
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Nuageux,800,377,0.190307,"[133.36938839157773, 145.10622021532942, 168.6...","[1314, 3819, 2962, 15665, 32156, 29125, 10206,...","[109.98722163219205, 57.693647903707074]"


In [57]:
# Calcul de l'entropie de la luminance globale

luminance_values = df['Luminance']
histogram, _ = np.histogram(luminance_values, bins=10, density=True)
luminance_entropy = entropy(histogram, base=2).round(2)

print("Luminance Entropy :",luminance_entropy)


Luminance Entropy : 2.71


In [58]:
# Calcul de l'entropie de la luminance par classe

luminance_entropy_by_label = df.groupby('Label')['Luminance'].apply(lambda x: entropy(np.histogram(x, bins=10, density=True)[0], base=2).round(2))

print("Luminance Entropy by Label:")
print(luminance_entropy_by_label)


Luminance Entropy by Label:
Label
Ensoleillé         2.95
Lever du soleil    2.74
Nuageux            2.87
Pluvieux           3.06
Name: Luminance, dtype: float64


In [59]:
luminance_entropy_df = pd.DataFrame(luminance_entropy_by_label)
luminance_entropy_df

,Luminance
Label,
Ensoleillé,2.95
Lever du soleil,2.74
Nuageux,2.87
Pluvieux,3.06


In [60]:
# Calcul de l'entropie de chaque canal RGB de maniere globale

df['RGB'] = df['RGB'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
rgb_df = pd.DataFrame(df['RGB'].tolist(), columns=['R', 'G', 'B'])

# Calculate the entropy for each RGB channel
rgb_entropy = {}
for channel in ['R', 'G', 'B']:
    hist, _ = np.histogram(rgb_df[channel], bins=10, density=True)
    rgb_entropy[channel] = entropy(hist, base=2).round(2)

rgb_entropy


{'R': 2.82, 'G': 2.83, 'B': 3.21}

In [61]:
# Calcul de l'entropie de chaque canal RGB par classe 

df['RGB'] = df['RGB'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Split RGB into separate columns
rgb_df = pd.DataFrame(df['RGB'].tolist(), columns=['R', 'G', 'B'])
rgb_df['Class'] = df['Label']  
class_entropy = {}

# Calculate the entropy for each channel within each class
for class_name, group in rgb_df.groupby('Class'):
    class_entropy[class_name] = {}
    for channel in ['R', 'G', 'B']:
        hist, _ = np.histogram(group[channel], bins=10, density=True)
        class_entropy[class_name][channel] = entropy(hist, base=2).round(2)

class_entropy


{'Ensoleillé': {'R': 2.89, 'G': 3.02, 'B': 2.42},
 'Lever du soleil': {'R': 2.99, 'G': 2.92, 'B': 2.88},
 'Nuageux': {'R': 2.87, 'G': 2.96, 'B': 2.97},
 'Pluvieux': {'R': 2.92, 'G': 2.95, 'B': 2.89}}

In [49]:
RGB_Enteropy_df = pd.DataFrame(class_entropy)
RGB_Enteropy_df = RGB_Enteropy_df.transpose()
RGB_Enteropy_df


,R,G,B
Ensoleillé,2.89,3.02,2.42
Lever du soleil,2.99,2.92,2.88
Nuageux,2.87,2.96,2.97
Pluvieux,2.92,2.95,2.89


In [63]:
# Calcul de l'entropie de chaque canal H,S de maniere globale

df['HSV'] = df['HSV'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
hsv_df = pd.DataFrame(df['HSV'].tolist(), columns=['H', 'S'])

# Calculate the entropy for each RGB channel
hsv_entropy = {}
for channel in ['H', 'S']:
    hist, _ = np.histogram(hsv_df[channel], bins=10, density=True)
    hsv_entropy[channel] = entropy(hist, base=2).round(2)

hsv_entropy

{'H': 2.95, 'S': 3.16}

In [64]:
# Calcul de l'entropie de chaque canal H,S par classe

df['HSV'] = df['HSV'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

hsv_df = pd.DataFrame(df['HSV'].tolist(), columns=['H', 'S'])
hsv_df['Label'] = df['Label']  

label_entropy = {}

for label, group in hsv_df.groupby('Label'):
    label_entropy[label] = {}
    for channel in ['H', 'S']:
        hist, _ = np.histogram(group[channel].dropna(), bins=10, density=True)
        if hist.sum() > 0:  
            label_entropy[label][channel] = entropy(hist, base=2).round(2)
        else:
            label_entropy[label][channel] = None  

label_entropy


{'Ensoleillé': {'H': 1.78, 'S': 2.86},
 'Lever du soleil': {'H': 2.73, 'S': 3.19},
 'Nuageux': {'H': 2.61, 'S': 2.65},
 'Pluvieux': {'H': 2.96, 'S': 2.27}}

In [52]:
HSV_Entropy_df = pd.DataFrame(label_entropy)
HSV_Entropy_df = HSV_Entropy_df.transpose()
HSV_Entropy_df


,H,S
Ensoleillé,1.78,2.86
Lever du soleil,2.73,3.19
Nuageux,2.61,2.65
Pluvieux,2.96,2.27


## Résultat final 
Interprétation détaillée sur le rapport d'analyse 

In [54]:
combined_df = pd.concat([RGB_Enteropy_df, HSV_Entropy_df, luminance_entropy_df], axis=1)
combined_df


,R,G,B,H,S,Luminance
Ensoleillé,2.89,3.02,2.42,1.78,2.86,2.95
Lever du soleil,2.99,2.92,2.88,2.73,3.19,2.74
Nuageux,2.87,2.96,2.97,2.61,2.65,2.87
Pluvieux,2.92,2.95,2.89,2.96,2.27,3.06
